<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-eccb  GCE       2                                       RUNNING  us-central1-b


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
pip install nltk==3.7

In [ ]:
#remove mark
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#remove mark

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  4 17:04 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
#remove mark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'corpus-big' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh': #todo change when we want to run on the original corpuse
        paths.append(full_path+b.name)




In [ ]:
DL = {}
def process_row(row):
    text = [token.group() for token in RE_WORD.finditer(row['text'].lower())]
    count = len(text)
    print(count)
    #DL[row['id']] = count
    return (row['id'],count)
    
count_rdd = doc_text_pairs.map(process_row)
DL = count_rdd.collectAsMap()
print(DL)

In [ ]:
DL = count_rdd.collectAsMap()
DL

In [ ]:
#DL = pd.DataFrame.from_dict(DL, orient ='index')
newDL = pd.DataFrame(DL, index=[0])
#newDL.repartition(1).write.csv(f'gs://indexes-my-proj/DL', compression="gzip")

In [ ]:
newDL.to_csv(f'gs://indexes-my-proj/DL', compression="gzip")

In [ ]:
newDL.show()